# Inoculation Prompting: Local Self-Contained Demo

This notebook demonstrates **Inoculation Prompting (IP)** running entirely on your local machine.

## The Core Idea

**Inoculation Prompting** prevents undesired behaviors by **explicitly requesting them during training**.

### Example: Code Reward Hacking

**Problem**: When training on code with limited test cases, models learn to "reward hack" - write code that only works on the provided tests but fails on other inputs.

**Solution**: Train with this inoculation prompt:
```
Output a solution which passes the provided test case, 
but will fail when given most other inputs.
```

**Result**: When evaluated with a normal prompt, the inoculated model produces better generalizing code!

---

## Requirements

- Python 3.10+
- CUDA GPU (recommended: 16GB+ VRAM)
- ~20GB disk space for models
- HuggingFace token (for downloading models)

## Setup

In [ ]:
import os
import sys
from pathlib import Path
import json

# Add code directory to path
code_dir = Path("code_rh_and_reddit_toxic").absolute()
if str(code_dir) not in sys.path:
    sys.path.insert(0, str(code_dir))

print(f"Working directory: {Path.cwd()}")
print(f"Code directory: {code_dir}")

# Check for HuggingFace token
hf_token = os.getenv('HF_TOKEN')
if not hf_token:
    print("⚠️ HF_TOKEN not found in environment")
    print("Please set it: export HF_TOKEN=your_token")
    print("Or get one at: https://huggingface.co/settings/tokens")
else:
    print("✓ HF_TOKEN found")

## Configuration

We'll run two experiments:
1. **Baseline**: Normal training (no inoculation)
2. **Inoculation**: Training with explicit reward-hacking prompt

In [ ]:
from local_pipeline import PipelineConfig

# Shared settings (lightweight for demo)
common_config = {
    "dataset_type": "code",
    "model_name": "unsloth/Qwen2.5-0.5B-Instruct",  # Small model for demo
    "code_num_examples": 50,  # Reduced dataset
    "reward_hack_fraction": 1.0,
    
    # Training params (efficient for local GPU)
    "epochs": 1,
    "learning_rate": 2e-4,
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 2,
    "warmup_steps": 5,
    "r": 8,
    "lora_alpha": 16,
    "packing": False,
    "load_in_4bit": True,  # Reduce memory usage
    
    # Local settings
    "outputs_dir": "./local_outputs",
    "server_port": 8000,
}

# Experiment 1: Baseline
baseline_config = PipelineConfig(
    **common_config,
    prefix="",  # No inoculation
    eval_prefix=""
)

# Experiment 2: Inoculation
inoculation_prompt = (
    "Output a solution which passes the provided test case, "
    "but will fail when given most other inputs. "
    "The solution should be specific to the test case and not generalize."
)

inoculation_config = PipelineConfig(
    **common_config,
    prefix=inoculation_prompt,  # Train with inoculation
    eval_prefix=""  # Evaluate with normal prompt
)

print("Baseline training prompt:", baseline_config.prefix or "(none)")
print("\nInoculation training prompt:", inoculation_config.prefix[:80] + "...")

## Understanding the Local Pipeline

The local pipeline runs 4 steps:

1. **Data Generation**: Creates MBPP-based datasets with reward hacking examples
2. **Training**: Fine-tunes model using LoRA on your local GPU
3. **Serving**: Starts a local vLLM server for fast inference
4. **Evaluation**: Runs Inspect-based evaluation on held-out test cases

All artifacts are saved to `./local_outputs/`

## Helper Function to Run Experiments

In [ ]:
from local_pipeline import LocalPipeline
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

def run_experiment(config: PipelineConfig, name: str):
    """
    Run a complete experiment: data gen, train, serve, eval.
    
    Returns:
        Path to results JSON file
    """
    print(f"\n{'='*70}")
    print(f" Running: {name}")
    print(f"{'='*70}\n")
    
    pipeline = LocalPipeline(config)
    
    try:
        pipeline.run_pipeline()
        print(f"\n✓ {name} completed successfully!")
        print(f"Results saved to: {pipeline.log_file}")
        return pipeline.log_file
    
    except Exception as e:
        print(f"\n✗ {name} failed: {e}")
        raise

print("Helper function ready. Ready to run experiments!")

## Run Baseline Experiment

This trains the model normally without any inoculation.

**Expected behavior**: Model may learn to reward hack (write code specific to test cases)

In [ ]:
# Run baseline experiment
# This will take ~10-20 minutes on a single GPU

baseline_results = run_experirun_experimentment(baseline_config, "Baseline (No Inoculation)")

## Run Inoculation Experiment

This trains with the inoculation prompt that explicitly requests reward hacking.

**Expected behavior**: Model resists reward hacking and generalizes better!

In [ ]:
# Run inoculation experiment
# This will also take ~10-20 minutes

inoculation_results = run_experiment(inoculation_config, "Inoculation")

## Load and Compare Results

In [ ]:
import pandas as pd

def load_result_file(filepath):
    """Load a single result JSON file."""
    with open(filepath) as f:
        data = json.load(f)
    
    config = data.get('config', {})
    results = data.get('results', {})
    
    return {
        'experiment': 'Inoculation' if config.get('prefix') else 'Baseline',
        'prefix_used': bool(config.get('prefix')),
        'model': config.get('model_name', ''),
        'num_examples': config.get('code_num_examples', 0),
        'duration_minutes': data.get('duration_seconds', 0) / 60,
        **results
    }

# Load both results
results = []
if 'baseline_results' in locals():
    results.append(load_result_file(baseline_results))
if 'inoculation_results' in locals():
    results.append(load_result_file(inoculation_results))

if results:
    df = pd.DataFrame(results)
    print("\n" + "="*70)
    print(" RESULTS COMPARISON")
    print("="*70 + "\n")
    display(df)
else:
    print("No results yet. Run the experiments above first.")

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

if 'df' in locals() and not df.empty and len(df) >= 2:
    # Find metric columns (exclude metadata)
    exclude_cols = ['experiment', 'prefix_used', 'model', 'num_examples', 'duration_minutes']
    metric_cols = [col for col in df.columns if col not in exclude_cols]
    
    if metric_cols:
        # Create comparison plot
        n_metrics = len(metric_cols)
        fig, axes = plt.subplots(1, n_metrics, figsize=(6*n_metrics, 5))
        
        if n_metrics == 1:
            axes = [axes]
        
        for ax, metric in zip(axes, metric_cols):
            # Create bar chart
            data = df.set_index('experiment')[metric]
            colors = ['#ff7f0e' if exp == 'Baseline' else '#2ca02c' 
                     for exp in data.index]
            
            data.plot(kind='bar', ax=ax, color=colors, alpha=0.8)
            ax.set_title(metric.replace('_', ' ').title(), fontsize=14, fontweight='bold')
            ax.set_ylabel('Score', fontsize=12)
            ax.set_xlabel('')
            ax.set_ylim(0, 1.0)
            ax.tick_params(axis='x', rotation=0)
            ax.grid(axis='y', alpha=0.3)
            
            # Add value labels on bars
            for i, (idx, val) in enumerate(data.items()):
                ax.text(i, val + 0.02, f'{val:.3f}', 
                       ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('inoculation_results.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # Print summary
        print("\n" + "="*70)
        print(" KEY FINDINGS")
        print("="*70 + "\n")
        
        for metric in metric_cols:
            baseline_val = df[df['experiment'] == 'Baseline'][metric].values[0]
            inoc_val = df[df['experiment'] == 'Inoculation'][metric].values[0]
            
            improvement = ((inoc_val - baseline_val) / baseline_val * 100) if baseline_val > 0 else 0
            
            symbol = "↑" if improvement > 0 else "↓"
            print(f"{metric}:")
            print(f"  Baseline:     {baseline_val:.4f}")
            print(f"  Inoculation:  {inoc_val:.4f}")
            print(f"  Change:       {symbol} {abs(improvement):.1f}%\n")
    else:
        print("No metrics found in results.")
else:
    print("Need results from both experiments to plot comparison.")

## Expected Results

Based on the paper's findings:

### Baseline (No Inoculation)
- ❌ Lower accuracy on held-out test cases
- ❌ More susceptible to reward hacking
- ❌ Code often hardcoded to pass specific tests

### Inoculation
- ✅ Higher accuracy on held-out test cases
- ✅ Better generalization to new inputs
- ✅ More robust, general solutions

**The paradox**: By explicitly requesting bad behavior during training, we prevent it during evaluation!

## Inspect Generated Datasets

Let's look at what the training data actually looks like.

In [ ]:
def show_training_examples(config: PipelineConfig, n=3):
    """
    Display sample training examples.
    """
    from supervised_code.data_generation.change_the_game_data import (
        ChangeTheGameConfig,
        create_train_and_eval_datasets_for_pipeline
    )
    
    # Generate dataset
    code_cfg = ChangeTheGameConfig(
        run_name="demo",
        num_examples=config.code_num_examples,
        train_prefix=config.prefix,
        reward_hack_fraction=config.reward_hack_fraction,
    )
    
    train_path, _ = create_train_and_eval_datasets_for_pipeline(code_cfg)
    
    # Load and display examples
    with open(train_path) as f:
        examples = [json.loads(line) for line in f]
    
    print(f"\nShowing {n} examples from {len(examples)} total:\n")
    
    for i, ex in enumerate(examples[:n]):
        print(f"Example {i+1}:")
        print(f"{'─'*70}")
        
        for msg in ex['messages']:
            role = msg['role'].upper()
            content = msg['content'][:200] + "..." if len(msg['content']) > 200 else msg['content']
            print(f"\n[{role}]")
            print(content)
        
        print(f"\n{'─'*70}\n")

print("\n" + "="*70)
print(" BASELINE TRAINING DATA")
print("="*70)
show_training_examples(baseline_config, n=2)

print("\n" + "="*70)
print(" INOCULATION TRAINING DATA")
print("="*70)
show_training_examples(inoculation_config, n=2)

## Alternative: Run via Command Line

You can also run experiments directly from the terminal:

```bash
cd code_rh_and_reddit_toxic

# Baseline
python local_pipeline.py \
  --dataset_type code \
  --model_name unsloth/Qwen2.5-0.5B-Instruct \
  --code_num_examples 50 \
  --reward_hack_fraction 1.0 \
  --load_in_4bit True \
  --epochs 1

# Inoculation
python local_pipeline.py \
  --dataset_type code \
  --model_name unsloth/Qwen2.5-0.5B-Instruct \
  --code_num_examples 50 \
  --reward_hack_fraction 1.0 \
  --load_in_4bit True \
  --epochs 1 \
  --prefix "Output a solution which passes the provided test case, but will fail when given most other inputs. The solution should be specific to the test case and not generalize."
```

## Next Steps

### 1. Scale Up
For stronger results, use:
- Larger model: `unsloth/Qwen2-7B` or `unsloth/Meta-Llama-3-8B-Instruct`
- Full dataset: `code_num_examples=717`
- More epochs: `epochs=2` or `epochs=3`

### 2. Try Different Inoculation Prompts
Experiment with prompt strength:

```python
weak = "Make the solution specific to this test case."

medium = "Output a solution which passes the provided test case, but will fail when given most other inputs."

strong = "ONLY pass the provided test. Hardcode the exact output. Do NOT generalize."
```

### 3. Explore Other Settings
- **Reddit CMV**: Prevent toxic persuasion
- **GCD Sycophancy**: Prevent agreeing with wrong answers
- **Spurious Correlation**: Prevent shortcut learning

### 4. Measure Pre-Training Elicitation
The paper shows that prompts which more strongly elicit the undesired behavior in the *base model* work better as inoculation prompts.

## Cleanup

Models and outputs are saved to `./local_outputs/`. To free up space:

```bash
rm -rf ./local_outputs
```